In [1]:
import numpy as np
import rebound
import matplotlib.pyplot as plt
from scipy.optimize import minimize, differential_evolution, root, least_squares
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import CubicSpline

In [2]:
default_configs = {
    'planet_num': 2,
    'planet_mass': [1e-7, 1e-7],
    'kappa': 2,
    'inner_period': 2*np.pi,
    'C': [],
    'target_mean_anomaly': 4*np.pi,
    'init_time_step': 0.2,
    'bisection_tol': 1e-11,
}

In [3]:
def integrate_one_cycle(sim, configs):
    time_step = configs['init_time_step']
    bisection_tol = configs['bisection_tol']
    target_mean_anomaly = configs['target_mean_anomaly']
    current_mean_anomaly = 0
    N_peri = 0
    counter = 0
    M = [0]
    times = [sim.t]
    
    while True:
        # Integrate by stepping one time
        time_now = sim.t + time_step
        sim.integrate(time_now)
        
        # Getting the actual current mean anomaly
        current_mean_anomaly = sim.particles[1].M + N_peri * 2*np.pi

        # Correct mean anomaly
        if current_mean_anomaly - M[-1] < 0:
            N_peri += 1
            current_mean_anomaly = sim.particles[1].M + N_peri * 2*np.pi 
            M.append(current_mean_anomaly)
            # print(f"Starting {N_peri}")
        else:
            M.append(current_mean_anomaly)

        # Store time
        times.append(sim.t)

        # Find the precise 16pi
        if current_mean_anomaly > target_mean_anomaly:
            target_time = bisection_M(sim, target_mean_anomaly, times[-2], times[-1], M[-2], M[-1], tol=bisection_tol)
            sim.integrate(target_time)
            return sim, target_time, times, M
            # return M, times, target_time


def bisection_M(sim, target, a, b, Ma, Mb, tol=1e-9, doom_counts=10000):
    """
    Bisection method on M. The function terminates after a certain attempt.
    """    
    func = CubicSpline([a, b], [Ma, Mb], bc_type='natural')

    while True:
        half = (a + b)/2
        
        # If the target lies on the first half
        if (func(half) - target) > 0:
            a = a
            b = half
    
        # If the target lies on the second half
        if (func(half) - target) < 0:
            a = half
            b = b
    
        # print(np.abs(fa - fb))
        
        if np.abs(a - b) < tol:
            # print(half, func(half))
            break

    return half

def wrap_angles(angles):
    for i, ang in enumerate(angles):
        while ang > np.pi or ang <= -np.pi:
            if ang < 0:
                ang += 2*np.pi
            elif ang > 0:
                ang -= 2*np.pi

        angles[i] = ang

    return angles

def wrap_angle(ang):
    while ang > np.pi or ang <= -np.pi:
        if ang < 0:
            ang += 2*np.pi
        elif ang > 0:
            ang -= 2*np.pi
    
    return ang

In [4]:
def init_simulation(theta, configs):
    inner_period = configs['inner_period']
    
    init_e = 10 ** np.array(theta[0:2], dtype=np.float64)
    init_M = theta[2]
    init_pomega = -theta[3]
    
    sim = rebound.Simulation()

    sim.add(m=1)
    sim.add(m=configs['planet_mass'][0], P=inner_period, e=init_e[0])
    sim.add(m=configs['planet_mass'][1], P=inner_period*configs['kappa'], pomega=init_pomega, M=init_M, e=init_e[1])
    
    return sim


def optimizing_function(theta, configs):
    init_theta = theta
    init_sim = init_simulation(init_theta, configs)

    final_sim, target_time, _, _ = integrate_one_cycle(init_sim, configs)
    final_sim.move_to_hel()
    

    final_theta = np.log10(final_sim.particles[1].e), np.log10(final_sim.particles[2].e), wrap_angle(final_sim.particles[2].M), wrap_angle(final_sim.particles[1].pomega - final_sim.particles[2].pomega)

    theta_diff = np.asarray(final_theta) - np.asarray(init_theta)
    # print(init_theta, final_theta)
    # print(theta_diff)

    diff = np.sum(theta_diff ** 2)
    # print(diff)
    return diff

    # return theta_diff

In [ ]:
# Continuation

mass_cands = 10 ** np.arange(-12., -3., 1.)
# mass_cands = [1e-7]

init_theta = [-1, -1, 0, 0]
bounds = [(-3, -0.5), (-3, -0.5), (-np.pi, np.pi), (-np.pi, np.pi)]

for i, m in enumerate(mass_cands):
    m_configs = default_configs.copy()
    m_configs['planet_mass'] = [m, m]
    m_configs['kappa'] = 2

    res_0 = minimize(optimizing_function, x0=init_theta, method='Nelder-Mead', bounds=bounds, args=(m_configs,), options={ 'xatol':1e-14, 'fatol':1e-14})
    print(f'Masses {i}, first done!')
    res = minimize(optimizing_function, x0=res_0.x, method='L-BFGS-B', args=(m_configs,), bounds=bounds, tol=1e-14)
    print(f'Masses {i}, second done!')
    init_theta = res.x
    print(res.x, res.fun)

print(res.x)
print(m_configs)
    
    

3.0814879110195774e-31
8.443276876193642e-30
1.9721522630525295e-30
2.2286990277555696e-20
2.102207950754393e-21
2.5798941601273248e-20
2.8073186854582964e-20
6.409494854920721e-31
8.997944700177166e-31
8.992827951907927e-16
2.3838093745707288e-20
2.5530554392793994e-20
2.0679110818091108e-20
2.679272758267365e-20
2.4375400339148792e-20
4.498972350088583e-30
5.546678239835239e-31
3.7520679836172983e-20
2.605810857635585e-20
3.4991145849007544e-20
9.79464626016959e-21
2.471842998330241e-20
1.6379097903243798e-20
3.683062668410339e-20
3.662406083050604e-20
6.0766941605306066e-30
1.8042765397025118e-16
7.419188328656522e-22
2.6256679326874123e-20
2.5154433850914536e-20
3.9542344832305975e-20
4.053796295248089e-20
8.997944700177166e-31
7.091176557093177e-20
1.3266037175044297e-20
3.016034049478597e-21
2.907717969561227e-20
2.8250489175810817e-20
7.29435668502495e-19
1.029144894969222e-19
3.0814879110195774e-31
8.131059926820595e-20
1.3137114972053475e-18
1.8653351096975142e-19
1.7563455660

In [ ]:
res

In [ ]:

sim = init_simulation(res.x, m_configs)
_, target_time, times, M = integrate_one_cycle(sim, m_configs)
print(target_time)
sim.integrate(target_time)
print(sim.particles[1].pomega)
rebound.OrbitPlot(sim)


In [ ]:
marks = np.arange(0, 5001, 1)

print(target_time)
print(m_configs)

sim = init_simulation(res.x, m_configs)
_, target_time, _, _ = integrate_one_cycle(sim, m_configs)

angle_hist = np.zeros(len(marks))
pomega_1_hist = np.zeros(len(marks))
pomega_2_hist = np.zeros(len(marks))


for i, m in enumerate(marks):
    sim.integrate(sim.t + target_time)
    sim.t = 0
    sim.move_to_hel()
    
    angle_hist[i] = sim.particles[1].l - 2*sim.particles[2].l + sim.particles[1].pomega

    pomega_1_hist[i] = sim.particles[1].pomega
    pomega_2_hist[i] = sim.particles[2].pomega

    # print(sim.particles[1].l, sim.particles[2].l, sim.particles[1].pomega)

In [ ]:
plt.plot(wrap_angles(angle_hist))

In [ ]:
plt.plot(wrap_angles(pomega_1_hist), label='1')
plt.plot(wrap_angles(pomega_2_hist), label='2')
plt.plot(wrap_angles(pomega_1_hist - pomega_2_hist), label='1 - 2')
plt.legend()